In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [72]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [71]:
def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

## Test Improvement

In [76]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

scal = StandardScaler()
x_train = scal.fit_transform(x_train)
x_test = scal.transform(x_test)

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1))



checkp = ModelCheckpoint(filepath='weights.hdf5', save_best_only=True)
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=400,
        batch_size=500,
        validation_data=(x_test, y_test),
        callbacks=cbs)

Train on 3567 samples, validate on 892 samples
Epoch 1/400
3567/3567 [==============================] - 0s - loss: 11.6307 - val_loss: 7.8656
Epoch 2/400
3567/3567 [==============================] - 0s - loss: 9.2503 - val_loss: 6.3352
Epoch 3/400
3567/3567 [==============================] - 0s - loss: 8.0292 - val_loss: 7.6219
Epoch 4/400
3567/3567 [==============================] - 0s - loss: 7.1863 - val_loss: 4.7713
Epoch 5/400
3567/3567 [==============================] - 0s - loss: 6.5798 - val_loss: 3.9618
Epoch 6/400
3567/3567 [==============================] - 0s - loss: 6.1210 - val_loss: 4.2506
Epoch 7/400
3567/3567 [==============================] - 0s - loss:

## Actual Training

In [81]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

scal = StandardScaler()
x_train = scal.fit_transform(x_train)
x_test = scal.transform(x_test)


model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1))

checkp = ModelCheckpoint(filepath='weights_final.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=400,
        batch_size=400,
        callbacks=cbs)

Epoch 1/400
4459/4459 [==============================] - 0s - loss: 16.5259     
Epoch 2/400
4459/4459 [==============================] - 0s - loss: 9.5369     
Epoch 3/400
4459/4459 [==============================] - 0s - loss: 8.3251     
Epoch 4/400
4459/4459 [==============================] - 0s - loss: 7.4224     
Epoch 5/400
4459/4459 [==============================] - 0s - loss: 6.6644     
Epoch 6/400
4459/4459 [==============================] - 0s - loss: 5.9002     

In [82]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred.csv',index=False)

48960/49342 [============================>.] - ETA: 0s  